In [1]:
from IPython.display import clear_output
from google.colab import drive
!pip install --upgrade openai==1.56.1
clear_output()
drive.mount('/content/drive')
%cd /content/drive/MyDrive/우리은행/similarity

Mounted at /content/drive
/content/drive/MyDrive/우리은행/similarity


In [3]:
from data_utils_py import (load_from_jsonl, save_to_jsonl, prompt_json,
                        gpt_batch_request, gpt_batch_status, gpt_result_file_save, gpt_func_test)
from jinja2 import Template
import os, random, json

def load_template(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        template_content = file.read()

    return Template(template_content)

In [119]:
sys_prompt_path = "prompt_templates_similarity/system_prompt.j2"
# advanced_lst_path = 'prompt_templates_similarity/advanced_lst.j2'
few_shot_prompt_path = 'prompt_templates_similarity/few_shot_format.j2'
few_shot_lst_path = 'prompt_templates_similarity/few_shot_lst.jsonl'
usr_prompt_path = "prompt_templates_similarity/user_prompt.j2"

sys_prompt_template = load_template(sys_prompt_path)
# advanced_lst_template = load_template(advanced_lst_path)
few_shot_prompt_template = load_template(few_shot_prompt_path)
few_shot_lst = load_from_jsonl(few_shot_lst_path)
usr_prompt_template = load_template(usr_prompt_path)

In [118]:
advanced_lst='''```
오피스텔 소유자는 무주택자로 간주합니다.
유주택자는 주택을 1채 이상 보유 중인 고객이며, 다주택자는 주택을 2채 이상 보유하고 있는 고객입니다.
우리WON주택대출은 고정금리 또는 혼합금리를 제공하지 않습니다.
'전세를 끼고 주택을 구매하는 경우'는 고객이 세입자에게 주택을 임차하는 목적으로 주택을 구입하는 경우를 뜻합니다.
대출신청부터 대출실행까지의 기간은 대출심사에 따라 달라질 수 있습니다. 정확한 대출심사기간은 부동산금융디지털센터(tel: 1599-9700)을 통해 문의 바랍니다.
n개월 혹은 n년 변동금리는 대출실행일 기준으로 n 기간마다 금리가 재조정됩니다.
생애최초 주택구매자는 세대 구성원 모두가 과거제 주택을 소유한 사실이 없는 자를 뜻하며, 구입자금목적의 대출한도는 6억원입니다.
경매주택의 경우에는 전자등기 처리가 불가하여 우리WON주택대출 취급이 불가합니다.
```'''

# for a in advanced_str.split('\n'):
#     print(a)
#     print('-='*20)

# save_to_jsonl(advanced_lst_path, advanced_str.split('\n'))

In [ ]:
few_shot_str = '''### question: 기한의 이익이 상실되었을 때, 언제부터 대출원금에 대한 연체이자가 부과되나요?
### doc: - ·  기한의  이익이 상실된 때에는 그 다음 날부터 대출원금에 대한 연체이자 를 내셔야 합니다.

### description: 질문은 기한의 이익 상실 시 연체이자 부과 시점을 묻고 있으며, 문단은 기한이익 상실 다음 날부터 연체이자를 내야 한다는 동의어 관계를 통해 질문에 직접적인 답변을 포함하고 있습니다.
### score: 100


### question: 만기 일시상환 방식의 위험 요소는 무엇인가요?
### doc: - · 대출금을 중도에 상환하는 경우 중도상환해약금 이 부과될 수 있습니다. 중도상환해약금은 대출의 잔여일수 및 상환금액에 따라 달라지며, 중도상환 시 향후 원리금 상환 부담금액이 감소하게 됩니다.\n-  발생 가능한 불이익에 관한 사항  \ṅ  \ṅ  \ṅ  \ṅ  \n 대출금 연체 시 대출원금 에 대한 연체이자 납부, 연체정보 등록과 같은 불이익이 발생할 수 있습니다   \n상세내용은 본 상품설명서의 [8.원리금 납부 연체에 따른 불이익] 항목을 참고하시기 바랍니다.

### description: 문단은 대출의 조기 상환 및 연체에 따른 불이익을 다루지만, 질문에서 묻는 만기 일시상환 방식의 핵심적인 위험 요소인 '만기 시 상환 부담'이나 '금리 변동 위험'을 포함하지 않아 직접적인 답변으로 보기 어렵습니다.
### score: 40


### question: 대출 실행 후 언제까지 청약 철회가 가능한가요?
### doc: "- · 대출계약의 해지 시 중도상환해약금이 부과될 수 있습니다. 중도상환해약금이 대출계약이 성립한 날부터 3년 이내의 기간 에만 부여됩니다.\n- · 대출신청 이후 고객에 의한 대출취소 또는 대출심사 거절 시 발생한 비용(등기설정·말소비용 등)에 대해서는 고객이 부담해야 합니다.\n- ◉ 은행은 고객님의 정확한 상환능력을 추론하기 위하여 DSR(총부채원리금상환비율)을 산출하여 심사에 활용할 수 있으며, 연소득 대비 연원리금 상환비율이 높은 경우 심사가 거절될 수 있습니다.

### description: 질문은 대출 실행 후 청약 철회 가능 기간을 묻고 있지만, 문단은 대출계약 해지 시 중도상환해약금 부과 기간과 대출취소 시 비용 부담에 대해 설명하고 있어 직접적인 답변을 포함하지 않습니다.
### score: 30


### question: 2억 원의 대출을 받을 때 인지세는 얼마를 내야 하나요?
### doc: - 인지세 은행과 고객이 각각 50%씩 부담 「인지세법」에따라대출약정시납부하는세금으 로대출금액별차등납부\n- ※5천만원이하 면제 5천만원초과 1억원이하 7만원 각각3만5천원\n- 1억원 초과 10억원 이하 15만원 각각 7만5천원  \n10억원 초과 35만원 각각 17만5천원

### description: 질문은 2억 원 대출 시 인지세 금액을 구체적으로 묻고 있으며, 문단은 대출 금액별 인지세 요율을 명확히 제시하고 있어 질문에 대한 직접적인 답변을 포함하고 있습니다.
### score: 95


### question: 담보권은 언제 행사될 수 있으며, 그 종류는 무엇인가요?
### doc: "※  질권, 저당권 등 주택담보 外 담보권 설정에 해당하지 않을 경우 설명 생략 가능  \n※  담보의 제공은 고객님의 재산상 손실 을  가져올 수도 있는 중요한 법률행위이므로 반드시 별도로 작성하시는 근저당권(근질권)설정계약서의 내용을 상세하게 확인하시기 바랍니다.  \n- ◉ 대출과 관련하여 담보물을 제공한 경우, 은행은 채권보전을 위해 제공받은 담보물에 담보권 (저당권·질권 등)을 설정할 수 있습니다.

### description: 문단은 담보권의 종류(저당권, 질권 등)에 대해 언급하고 있지만, 담보권이 언제 행사될 수 있는지에 대한 명확한 설명은 포함하지 않아 질문에 대한 완전한 답변이 되지 않습니다.
### score: 50


### question: 오피스텔담보대출 신청 자격 요건은 무엇인가요?
### doc: - 해당상품은 우리은행 의대출상품으로20250113 기준으로 작성되었으며 내부통제 기준에 따 라심의절차를준수하여승인받은내용입니다\n- 일반금융소비자는금융소비자법제19조제1항에따라은행으로부터충분한설명을받을권리가있 으며 대출상품을계약하기전자세한내용은상품설명서및약관을읽어보시길권장합니다\n- 대출가능여부및최종대출금액은고객신용도 소득 담보방식 보유중인다른대출등에따라달라질 수있습니다\n- 자금조달비용의상승및금융당국의정책변화등금융시장변화가있을경우대출금리 대출한도등 대출조건이변경될수있습니다\n- 대출약정조건에따라대출이자또는원금을약정한날에납입 상환 하지않을경우연체이자율이적 용되며납부해야할원리금이일정기간이상연체될경우에계약만료기한이도래기전에모든원리금 을변제해야할의무가발생할수있습니다\n- 고객님의신용도 총부채상환비율 DTI 및 당행 심사기준에 따라 대출금액에 제한이 있을 수 있습니 다\n- 대출금의만기경과 이자납입일지연 기한의이익상실시다음의불이익이발생할수있습니다\n- ①대출잔액에대하여연체이자를납부하셔야합니다\n- ②연체기간에따라「신용정보의이용및보호에관한법률」 신용정보관리규약」에근거 신용도 판단정보 연체정보 대위변제및대지급정보 부도정보 금융질서문란정보등 등록자로등재되어금 융거래시불이익을받을수있습니다\n- ③보유중인예금과의상계 담보목적물처분 발견재산에대한보전조치등의법적절차가진행될 수있습니다\n- ④신용관리대상자 연체이력보유자 신용거래정보부족등여신부적격자로 분류되어대출취급이제한될수있습니다\n- 대출상환능력에비해대출금 신용카드사용액이과도할경우신용점수가하락할수있으며 신 용점수하락으로금융거래와관련된불이익이발생할수있습니다\n- 금리인하요구권 본인의신용상태에현저한변동이있는경우 직장변동 소득증가 자산증가 부채감소 신용등급상승등 증빙자료를제출하여금리변경을요구할수있습니다\n- 대출계약철회권 대출실행일로부터14일이내에신청가능하며 원금 이자 기타은행이부담한 비용등을반환해야철회가능합니다\n- 위법계약해지권 은행이 금융소비자 보호에 관한 법률 주1) 을위반하여계약을체결한경우 금융소비자는해당계약체결에대한위반사항을안날로부터1년이내 단 계약체결일로부터5 년이내이고계약이종료하기전일것 에서면등으로해당계약의해지를요구할수있습니다 은행은그해지를요구받은날로부터10일이내에금융소비자에게수락여부를통지하거나또 는거절사유와함께거절을통지하여야하고 만일정당한사유없이금융소비자의계약해지요 구를따르지않을경우금융소비자는해당계약을해지할수있으며 이에따라계약이해지될 경우은행은금융소비자에게수수료 위약금등비용을요구할수없습니다  \n→ 주1) 부적합한 금융상품을 권유한 경우(제17조 제3항), 적정하지 않은 금융상품에 대 해 부적정하다는 사실을 알리지 않은 경우(제18조 제2항), 금융상품에 대해 거짓 또는 왜곡하여 설명하거나 중요한 사항에 대한 설명을 누락한 경우 (제19조 제1항,3항), 불공 정영업행위의 금지(제20조 제1항), 부당권유행위의 금지(제21조)  \n- 자료열람요구권 본상품은자료열람요구권신청이가능합니다\n- 만기도래시거래실적 신용등급 개인신상 이직및퇴사 소득등 변동등에따라대출금리가 변동되거나 대출금의일부또는전액을상환하셔야할수도있습니다\n- 기타문의사항은가까운우리은행영업점또는고객센터 ☎1599 5000 1588 5000 로 문의 하시기바랍니다\n- 상기내용은우리은행의여신정책에따라달라질수있습니다  \n준법감시인심의필2025 0270 2025 01 10 2026 12 31

### description: 문단은 대출 가능 여부가 신용도, 소득, 담보 방식 등에 따라 결정된다고 설명하지만, 오피스텔담보대출 신청 자격 요건을 구체적으로 명시하지 않아 직접적인 답변으로 보기 어렵습니다.
### score: 45


### question: 대출금리 산정방식 및 결정요인을 이해하는 것이 왜 중요할까요?
### doc:  \ṅ  \n※아래 주요내용을 자필 로 확인 및 작성하여 주시기 바랍니다.  \n- [<Paragraph children=[<RawText children='[수수료 등 비용부담 사항] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[금융소비자의 권리] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[대출금리 산정방식 및 결정요인] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[상환방법 등에 관한 사항] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[담보권 설정에 관한 사항] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[원리금 납부 연체에 따른 불이익] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[계약기간 및 연장사항] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- [<Paragraph children=[<RawText children='[대출계약이 신용에 미치는 영향] 설명에 대한 내용을 이해하고 확인하셨나요? ⇨ ① 예 / ② 아니오'>]>]\n- 설명에 대한 내용을 이해하고 확인하셨나요?\n- [<Paragraph children=[<RawText children='[기타 유의사항] ⇨ ① 예 / ② 아니오'>]>]

### description: 문단은 대출금리 산정방식 및 결정요인에 대한 설명을 고객이 이해하고 확인했는지를 확인하며 질문과 일치하는 키워드가 포함되어 있으나, 해당 개념의 중요성에 대한 직접적인 설명이나 이유를 제공하지 않아 질문에 대한 답변이 되지 않습니다.
### score: 20


### question:고정금리 방식과 변동금리 방식의 주요 차이점은 무엇인가요?
### doc:- · 고객님께서 선택하신 상환방법은 [                  ] 입니다.\n- · 동일한  금리·한도로  대출을  진행하더라도  상환방법에  따라  만기까지  부담해야하는  총  원리금 부담액이 달라질 수 있습니다. 하단의 예시를 참고하시기 바랍니다.\n- ◉ 상환방법별 원리금 부담액 예시 (1억원을 대출기간 5년으로 하여 연 5%로 대출받은 경우)  \n※  이해를  돕기  위해  단순하게  계산한  예시이며,  실제  납부  원리금은  금리적용방식, 상환주기 등에 따라 달라질 수 있습니다.

### description: 질문은 고정금리와 변동금리 방식의 차이점을 묻고 있으며, 문단은 상환 방법에 따른 원리금 부담액 예시를 제공으로, 금리 방식에 대한 설명이 없어 질문과 연관된 답변을 제공하지 않습니다.
### score: 0
'''

for a in few_shot_str.split('\n\n\n'):
    print(a)
    print('-='*20)

save_to_jsonl(few_shot_lst_path, few_shot_str.split('\n\n\n'))

In [102]:
role = "당신은 금융업계 전문가입니다."
task_type = "사용자가 제공하는 질문과 문단을 분석하여, 질문과 문단 간 유사도를 평가한 데이터를 아래 형식에 맞게 생성하세요"
guideline='''1. **description:**
   - 질문과 문단의 핵심 키워드 및 문맥적 관련성을 고려하여, 유사도 평가 근거를 한 문장으로 요약합니다.
2. **score:**
   - 0부터 100까지의 정수로 유사도를 평가합니다.
   - **평가 기준:**
     - **100:** 완벽한 일치 (질문과 문단이 동일한 정보를 전달함)
     - **80~99:** 높은 관련성 (질문에 대한 명확한 답변을 포함하지만 일부 세부사항이 다를 수 있음)
     - **60~79:** 중간 정도의 관련성 (부분적으로 관련된 정보 포함)
     - **30~59:** 낮은 관련성 (일부 개념적 연관성 있으나 직접적인 답변 아님)
     - **0~29:** 거의 관련 없음'''
consideration='''- 질문과 문단에서 핵심 키워드를 추출하여 비교하세요.
- 문맥적 관련성과 표현 방식의 유사성을 중점적으로 평가하세요.
- 불필요한 CoT(Chain-of-Thought) 과정을 생략하고, 간단명료하게 답변하세요.
- 질문에 직접적으로 연관된 정보만 반영하여 평가하시기 바랍니다.'''


In [103]:
import pandas as pd
import json

df_filename = "docling_rag_res_0123.jsonl"
df_lst = load_from_jsonl(df_filename)
df_lst[0]

{'question': '대출금을 만기일에 상환하지 않으면 어떤 연체이자를 부담하게 되나요?',
 'doc_id': 1,
 'doc': '- 1 대출금을 만기일에 상환하지 아니한 때 대출잔액에 대하여 만기일의 다음날부터 상환일 전일까지 연체이자를납부하셔야합니다\n- 2 대출금의 기한의 이익이 상실된 때 대출잔액에 대하여 기한의 이익이 상실된 다음날부터 상환일 전일까지연체이자를납부하셔야합니다\n- 3 이자를 납부하기로 약정한 날에 납부하지 아니한 때 이자를 납부하여야 할 날의 다음날부터 1개 월해당일까지는납부하여야할이자에대하여 그이후에는대출잔액에대하여이자납부전일까지 연체이자를납부하셔야합니다\n- 4 분할상환금 또는 분할상환원리금 을 납부하기로 한 날에 상환하지 아니한 때 1회 지체시에는 지 연된분할상환금 또는분할상환원리금 에대하여 연속2회이상지체시에는대출잔액에대하여분 할상환금 또는 분할상환원리금 납입 전일까지 연체이자를 납부하셔야 합니다',
 'similarity': 0.9051367044448853}

In [104]:
question_lst = []
doc_list = []
for i, df in enumerate(df_lst):
    question = df['question']
    doc = df['doc']
    question_lst.append(question)
    doc_list.append(doc)

In [16]:
# 실제 question 및 doc 데이터 입력 시 아래의 코드 실행

# question_filename = "docling_question.jsonl"
# question_lst = load_from_jsonl(question_filename)

# doc_filename = "docling_rag.jsonl"
# doc_lst = load_from_jsonl(doc_filename)

In [127]:
def match_generator_product(question_lst, doc_lst):
  res_lst = []
  for i_q, question in enumerate(question_lst):
    for i_d, doc in enumerate(doc_list):
      id = 'sim_{}_{}'.format(i_q + 1, i_d + 1)
      few_shot = '\n\n\n'.join(random.sample(few_shot_lst, k=6))
      few_shot_prompt = few_shot_prompt_template.render(few_shot_sample=few_shot)
      sys_prompt = sys_prompt_template.render(role=role, task_type=task_type, guideline=guideline, consideration=consideration,
                                              advanced=advanced_lst, few_shot=few_shot_prompt)
      usr_prompt = usr_prompt_template.render(question=question, doc=doc)
      json_prompt = prompt_json(id, sys_prompt, usr_prompt, model_id='gpt-4o', max_tokens=3000)
      res_lst.append(json_prompt)

    return res_lst

In [128]:
res_lst = match_generator_product(question_lst, doc_list)

In [129]:
res_lst[1]

{'custom_id': 'sim_1_2',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o',
  'messages': [{'role': 'system',
    'content': '**역할:**\n당신은 금융업계 전문가입니다.\n\n**업무:**\n사용자가 제공하는 질문과 문단을 분석하여, 질문과 문단 간 유사도를 평가한 데이터를 아래 형식에 맞게 생성하세요\n\n\n**응답 데이터 형식:**\n```\n### description:\n### score:\n```\n\n**데이터 생성 지침:**\n1. **description:**  \n   - 질문과 문단의 핵심 키워드 및 문맥적 관련성을 고려하여, 유사도 평가 근거를 한 문장으로 요약합니다.\n2. **score:**  \n   - 0부터 100까지의 정수로 유사도를 평가합니다.  \n   - **평가 기준:**  \n     - **100:** 완벽한 일치 (질문과 문단이 동일한 정보를 전달함)  \n     - **80~99:** 높은 관련성 (질문에 대한 명확한 답변을 포함하지만 일부 세부사항이 다를 수 있음)  \n     - **60~79:** 중간 정도의 관련성 (부분적으로 관련된 정보 포함)  \n     - **30~59:** 낮은 관련성 (일부 개념적 연관성 있으나 직접적인 답변 아님)  \n     - **0~29:** 거의 관련 없음\n\n**추가 고려사항:**\n- 질문과 문단에서 핵심 키워드를 추출하여 비교하세요.  \n- 문맥적 관련성과 표현 방식의 유사성을 중점적으로 평가하세요.  \n- 불필요한 CoT(Chain-of-Thought) 과정을 생략하고, 간단명료하게 답변하세요.  \n- 질문에 직접적으로 연관된 정보만 반영하여 평가하시기 바랍니다.\n\n**추가 정보:**\n```\n오피스텔 소유자는 무주택자로 간주합니다.\n유주택자는 주택을 1채 이상 보유 중인 고객이며, 다주택자는

In [130]:
idx = 1
print(res_lst[idx]['body']['messages'][0]['content'])
print()
print(res_lst[idx]['body']['messages'][1]['content'])

**역할:**
당신은 금융업계 전문가입니다.

**업무:**
사용자가 제공하는 질문과 문단을 분석하여, 질문과 문단 간 유사도를 평가한 데이터를 아래 형식에 맞게 생성하세요


**응답 데이터 형식:**
```
### description:
### score:
```

**데이터 생성 지침:**
1. **description:**  
   - 질문과 문단의 핵심 키워드 및 문맥적 관련성을 고려하여, 유사도 평가 근거를 한 문장으로 요약합니다.
2. **score:**  
   - 0부터 100까지의 정수로 유사도를 평가합니다.  
   - **평가 기준:**  
     - **100:** 완벽한 일치 (질문과 문단이 동일한 정보를 전달함)  
     - **80~99:** 높은 관련성 (질문에 대한 명확한 답변을 포함하지만 일부 세부사항이 다를 수 있음)  
     - **60~79:** 중간 정도의 관련성 (부분적으로 관련된 정보 포함)  
     - **30~59:** 낮은 관련성 (일부 개념적 연관성 있으나 직접적인 답변 아님)  
     - **0~29:** 거의 관련 없음

**추가 고려사항:**
- 질문과 문단에서 핵심 키워드를 추출하여 비교하세요.  
- 문맥적 관련성과 표현 방식의 유사성을 중점적으로 평가하세요.  
- 불필요한 CoT(Chain-of-Thought) 과정을 생략하고, 간단명료하게 답변하세요.  
- 질문에 직접적으로 연관된 정보만 반영하여 평가하시기 바랍니다.

**추가 정보:**
```
오피스텔 소유자는 무주택자로 간주합니다.
유주택자는 주택을 1채 이상 보유 중인 고객이며, 다주택자는 주택을 2채 이상 보유하고 있는 고객입니다.
우리WON주택대출은 고정금리 또는 혼합금리를 제공하지 않습니다.
'전세를 끼고 주택을 구매하는 경우'는 고객이 세입자에게 주택을 임차하는 목적으로 주택을 구입하는 경우를 뜻합니다.
대출신청부터 대출실행까지의 기간은 대출심사에 따라 달라질 수 있습니다. 정확한 대출심사기간은 부동산금융디지털센터

In [131]:
save_to_jsonl('data/similarity_product.jsonl', res_lst)

In [ ]:
import openai

api_key = ''
client = openai.OpenAI(api_key=api_key)

id = gpt_batch_request('data/similarity_product.jsonl', client=client)

batch_id : batch_678a0eca14948190a1c778fdf63a6d5b


In [ ]:
print(client.batches.retrieve(id).status)
print(client.batches.retrieve(id).request_counts)

completed
BatchRequestCounts(completed=217, failed=0, total=217)


In [ ]:
gpt_result_file_save(id, 'reesult/similarity_product_result.jsonl', client=client)

complete and save


In [ ]:
# 아래부터는 기존 코드, 방향성에 따라 추후 수정 예정

In [ ]:
{'id', id,
 'question': ,
 'answer':}

In [ ]:
similarity_product_result_data = load_from_jsonl('result/similarity_product_result.jsonl')
print(len(similarity_product_result_data))

In [ ]:
similarity_product_result_data[0]

In [ ]:
import pandas as pd
import re

res = []
for data in similarity_product_result_data:
    qa_lst = data['response']['body']['choices'][0]['message']['content'].replace('```\n', '').replace('\n```', '')
    qa_lst = [x.strip() for x in qa_lst.split('### question:') if x.strip()!='']
    for c, qa in enumerate(qa_lst):
        id = '{}_{}'.format(data['custom_id'], c)
        question = qa.split('### question:')[0].split('\n### answer:')[0].replace('### question:', '').strip()
        answer = qa.split('\n### answer:')[1].strip()

        question = re.sub(r'^"|"$', '', question)
        answer = re.sub(r'^"|"$', '', answer)

        if (len(question)==0) or (len(answer)==0):
            break
        res.append({'id': id, 'question': question, 'answer': answer})


print(len(res))

420


In [ ]:
# save_to_jsonl('dictQA_data/acronyms.jsonl', res)